# 任务说明
服饰图像描述,训练一个模型,对输入的服饰图片,输出描述信息，我们实现的模型有以下三个实现：
- ARCTIC，一个典型的基于注意力的编解码模型
- 视觉Transformer (ViT) + Transformer解码器
- 网格/区域表示、Transformer编码器+Transformer解码器
同时也实现三种测评方法进行测评：
- BLEU (Bilingual Evaluation Understudy)
- SPICE (Semantic Propositional Image Caption Evaluation): 
- CIDEr-D (Consensus-based Image Description Evaluation)

# 实验数据
数据集使用的是 DeepFashion-MultiModal (https://github.com/yumingj/DeepFashion-MultiModal), 仅用到image和textual descriptions ，数据集划分为40k+行数据的训练集和2k+行数据的测试集，`train_captions.json`和`test_captions.json`分别对应训练集和测试集的图片与描述信息的键值对应

In [ ]:
img_path = f'data/deepfashion-multimodal/images'
def cap_to_wvec(vocab,cap):#将文本描述转换成向量
    cap.replace(",","")
    cap.replace(".","")
    cap=cap.split()
    res=[]
    for word in cap:
        if word in vocab.keys():
            res.append(vocab[word])
        else: #不在字典的词
            res.append(vocab['<unk>'])
    return res
def wvec_to_cap(vocab,wvec):#将向量转换成文本描述
    res=[]
    for word in wvec:
        for key,value in vocab.items():
            if value==word and key not in ['<start>','<end>','<pad>','<unk>']:
                res.append(key)
    res=" ".join(res)
    return res
def wvec_to_capls(vocab,wvec):#将向量转换成文本描述
    res=[]
    for word in wvec:
        for key,value in vocab.items():
            if value==word and key not in ['<start>','<end>','<pad>','<unk>']:
                res.append(key)
    return res
class ImageTextDataset(Dataset):
    def __init__(self, dataset_path, vocab_path, split, captions_per_image=1, max_len=93, transform=None):

        self.split = split
        assert self.split in {'train', 'test'}
        self.cpi = captions_per_image
        self.max_len = max_len

        # 载入数据集
        with open(dataset_path, 'r') as f:
            self.data = json.load(f) #key是图片名字 value是描述
            self.data_img=list(self.data.keys())
        # 载入词典
        with open(vocab_path, 'r') as f:
            self.vocab = json.load(f)

        # PyTorch图像预处理流程
        self.transform = transform

        # Total number of datapoints
        self.dataset_size = len(self.data_img)

    def __getitem__(self, i):
        # 第i个文本描述对应第(i // captions_per_image)张图片
        img = Image.open(img_path+"/"+self.data_img[i]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        c_vec=cap_to_wvec(self.vocab,self.data[self.data_img[i]])
        #加入起始和结束标志
        c_vec = [self.vocab['<start>']] + c_vec + [self.vocab['<end>']]
        caplen = len(c_vec)
        caption = torch.LongTensor(c_vec+ [self.vocab['<pad>']] * (self.max_len + 2 - caplen))
        
        return img, caption, caplen
        
    def __len__(self):
        return self.dataset_size
def mktrainval(data_dir, vocab_path, batch_size, workers=1):
    train_tx = transforms.Compose([
        transforms.Resize(256), # 重置图像分辨率
        transforms.RandomCrop(224), # 随机裁剪
        transforms.ToTensor(), # 转换成Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # 标准化--三个参数为三个通道的均值和标准差
    ])
    val_tx = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    train_set = ImageTextDataset(os.path.join(data_dir, 'train_captions.json'), vocab_path, 'train',  transform=train_tx)
    test_set = ImageTextDataset(os.path.join(data_dir, 'test_captions.json'), vocab_path, 'test', transform=val_tx)

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
    
    test_loader = torch.utils.data.DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=workers, pin_memory=True, drop_last=False)

    return train_loader, test_loader    
train_loader,test_loader=mktrainval(data_dir='data/deepfashion-multimodal',\
                                        vocab_path='data/deepfashion-multimodal/vocab.json',\
                                        batch_size=3,workers=0) 
#workers=0 是因为ipynb不支持多线程

# 实验环境
- Python  3.9.16
- 主要依赖库
  - torch 
  - torchvision
  - nltk 
实际使用py库情况如下

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence # 压紧填充序列
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.models import ResNet101_Weights
from nltk.translate.bleu_score import corpus_bleu # BLEU评价指标
import numpy as np
import json
from torch.utils.data import Dataset
import os
from PIL import Image
from collections import Counter,defaultdict
from argparse import Namespace 


# 所用的方法或模型

## 评估方法

### BLEU (BiLingual Evaluation Understudy)
- BLUE是比较常用的评估指标之一，也是我们默认指标，需要注意的是，再调用计算BLEU值之前，要先将文本中人工添加的文本开始符、结束符和占位符去掉，其公式如下， 实际代码中我们借助nltk库进行实现
$$BLEU = \sum_{n=1}^k w_n \frac{ngram_{sys}(n)}{ngram_{ref}(n)}$$
其中：
  - n 是 n-gram 的阶数，取值范围为 1 到 4。
  - wn 是 n-gram 的权重，通常取均匀权重。
  - ngramsys(n) 是机器翻译结果中的 n-gram 数量。
  - ngramref(n) 是参考翻译中的 n-gram 数量。
  BLEU 的得分范围为 0 到 1。得分越高，表示机器翻译结果与参考翻译越相似。
  - 优点：容易计算
  - 缺点:
    - 没有考虑n-gram的顺序
    - 平等对待所有的n-gram
    - 衡量的是句子之间的流畅性而非语义相似度
### CIDEr-D (Consensus-based Image Description Evaluation)
- 是CIDEr的改进，对于动词原形和名词匹配成功的问题，CIDEr-D不再取词根
其用了一种折扣函数来降低长句子对评分的影响，增加了惩罚生成句子和参考句子的长度差别的权重，并且通过对n-gram计数的截断操作不再计算生成句子中出现次数超过参考句子的n-gram,
从而减少了重复单词对评分的影响，其实也是计算1到4 gram的结果的平均值，其公式如下
$$C I D E r - D _ { n } ( c _ { i } , S _ { i } ) = \frac { 1 0 } { m } \sum _ { j } e ^ { - \frac { -( i ( c _ { i } ) - l ( s _ { i j } ) ) ^ { 2 } } { 2 \sigma ^ { 2 } } } \times \frac { \min ( g ^ { n } ( c _ { i } ) , g ^ { n } ( s _ { i j } ) ) \cdot g ^ { n } ( s _ { i j } ) } {| | g ^ { n } ( c _ { i } ) | | | g ^ { n } ( s _ { i j } ) || } $$
- 优点：
  - CIDEr引入了TF-IDF为n-gram进行加权，这样就避免评价候选句子时因为一些常见却不够有信息量的n-gram打上高分
- 缺点：
  - CIDEr取词根的操作会让一些动词的原型和名词匹配成功
  - 高置信度的词重复出现的长句的CIDEr得分也很高
### SPICE (Semantic Propositional Image Caption Evaluation): 
- 是以名词为中心的度量，是以图的语义表示来编码图像描述中的对象、属性和关系
首先要将候选句子和参考句子集转化为场景图
然后比较候选句子和参考句子集中元组的precision、recall，最终计算出F1 score
公式如下
$$SPICE = \sum_{i=1}^m \frac{1}{|S_i|} \sum_{j=1}^n \frac{s_{ij}}{|R_i|}
$$
  - m 是图像描述的数量。
  - n 是图像描述中的对象、属性和关系的数量。
  - Si 是图像描述 i 中的对象、属性和关系。
  - Ri 是参考图像描述 i 中的对象、属性和关系。
  - sij 是图像描述 i 中的对象、属性和关系 j 与参考图像描述 i 中的对象、属性和关系 j 的相似度
- 优点：
  - 在语义而非n-gram层级度量
  - 每个句子映射到场景图后可以从中提取出模型关于某些关系或者属性的识别能力
- 缺点
  - 缺少n-gram来度量句子的流畅性
  - 度量的准确性受到场景图解析器的制约

使用代码如下，在evaluate的时候调用,接受cands, refs返回对应评估分数

In [4]:
def cider_d(reference_list, candidate_list, n=4):
    def count_ngrams(tokens, n):
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def compute_cider_d(reference_list, candidate_list, n):
        cider_d_scores = []
        for refs, cand in zip(reference_list, candidate_list):
            cider_d_score = 0.0
            for i in range(1, n + 1):
                cand_ngrams = count_ngrams(cand, i)
                ref_ngrams_list = [count_ngrams(ref, i) for ref in refs]

                total_ref_ngrams = [ngram for ref_ngrams in ref_ngrams_list for ngram in ref_ngrams]

                count_cand = 0
                count_clip = 0

                for ngram in cand_ngrams:
                    count_cand += 1
                    if ngram in total_ref_ngrams:
                        count_clip += 1

                precision = count_clip / count_cand if count_cand > 0 else 0.0
                recall = count_clip / len(total_ref_ngrams) if len(total_ref_ngrams) > 0 else 0.0

                beta = 1.0
                f_score = (1 + beta**2) * precision * recall / (beta**2 * precision + recall) if precision + recall > 0 else 0.0

                cider_d_score += f_score

            cider_d_score /= n
            cider_d_scores.append(cider_d_score)

        return cider_d_scores

    reference_tokens_list = reference_list
    candidate_tokens_list = candidate_list

    scores = compute_cider_d(reference_tokens_list, candidate_tokens_list, n)

    return np.mean(scores)
def spice(reference_list, candidate_list, idf=None, beta=3):
    def tokenize(sentence):
        return sentence.lower().split()

    def count_ngrams(tokens, n):
        ngrams = []
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i:i+n])
            ngrams.append(ngram)
        return ngrams

    def compute_spice_score(reference, candidate, idf, beta):
        reference_tokens = reference
        candidate_tokens = candidate

        reference_ngrams = [count_ngrams(reference_tokens, i) for i in range(1, beta + 1)]
        candidate_ngrams = [count_ngrams(candidate_tokens, i) for i in range(1, beta + 1)]

        precision_scores = []
        recall_scores = []

        for i in range(beta):
            common_ngrams = set(candidate_ngrams[i]) & set(reference_ngrams[i])

            precision = len(common_ngrams) / len(candidate_ngrams[i]) if len(candidate_ngrams[i]) > 0 else 0.0
            recall = len(common_ngrams) / len(reference_ngrams[i]) if len(reference_ngrams[i]) > 0 else 0.0

            precision_scores.append(precision)
            recall_scores.append(recall)

        precision_avg = np.mean(precision_scores)
        recall_avg = np.mean(recall_scores)

        spice_score = (precision_avg * recall_avg) / (precision_avg + recall_avg) if precision_avg + recall_avg > 0 else 0.0

        if idf:
            spice_score *= np.exp(np.sum([idf[token] for token in common_ngrams]) / len(candidate_tokens))

        return spice_score

    if idf is None:
        idf = {}

    spice_scores = []

    for reference, candidate in zip(reference_list, candidate_list):
        spice_score = compute_spice_score(reference, candidate, idf, beta)
        spice_scores.append(spice_score)

    return np.mean(spice_scores)
def get_BLEU_score(cands, refs): #获取BLEU分数
    pasmultiple_refs = []
    for idx in range(len(refs)):
        multiple_refs.append(refs[(idx//1)*1 : (idx//1)*1+1])#每个候选文本对应cpi==1条参考文本
    bleu4 = corpus_bleu(multiple_refs, cands, weights=(0.25,0.25,0.25,0.25))
    return bleu4
def get_CIDER_D_score(cands, refs): #获得CIDER-D分数
    refs_ = [wvec_to_capls(model.vocab,ref) for ref in refs]
    cands_ = [wvec_to_capls(model.vocab,cand) for cand in cands]
    return cider_d(refs_, cands_)
def get_SPICE_score(cands, refs): #获得SPICE分数
    refs_ = [wvec_to_cap(model.vocab,ref) for ref in refs]
    cands_ = [wvec_to_cap(model.vocab,cand) for cand in cands]
    return spice(refs_, cands_)

## 模型定义
### ARCTIC，一个典型的基于注意力的编解码模型

### 视觉Transformer (ViT) + Transformer解码器

### 网格/区域表示、Transformer编码器+Transformer解码器

# 实验结果

# 实验结果分析

# 总结